In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

train_origin = pd.read_csv('./data/train_data.csv')
train = pd.read_csv('./data/FE_Housing_train.csv')
test = pd.read_csv('./data//FE_Housing_test.csv')

In [3]:
train_origin.shape

(41440, 51)

In [7]:
train.head()

,ID,area,rentType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,...,landMeanPrice_buildYear_cluster_mean,totalWorkers_buildYear_cluster_mean,newWorkers_buildYear_cluster_mean,residentPopulation_buildYear_cluster_mean,lookNum_buildYear_cluster_mean,trainsportNum_buildYear_cluster_mean,all_schoolNum_buildYear_cluster_mean,all_hospitalNum_buildYear_cluster_mean,all_mall_buildYear_cluster_mean,otherNum_buildYear_cluster_mean
0,100309852,68,1,1,16,6,0,50,0,63,...,0.000000,193780.461538,7624.692308,241618.461538,3.538462,6.252782,3.636850,2.980385,1.500007,4.188488
1,100307942,125,1,0,14,6,2,129,1,48,...,763.659060,49605.553684,4658.835789,296148.147368,1.846316,5.125237,3.543342,3.095838,1.816174,4.650882
2,100307764,132,1,1,32,6,0,178,1,49,...,458.530752,60515.231141,2490.489439,342397.438141,1.286059,7.775975,3.642007,3.752196,2.400410,6.286430
3,100306518,57,2,0,17,6,3,312,1,50,...,458.530752,60515.231141,2490.489439,342397.438141,1.286059,7.775975,3.642007,3.752196,2.400410,6.286430
4,100305262,129,1,1,2,6,1,1256,2,43,...,458.530752,60515.231141,2490.489439,342397.438141,1.286059,7.775975,3.642007,3.752196,2.400410,6.286430


In [8]:
Y_train = train_origin['tradeMoney']
X_train = train.drop('ID', axis=1)

In [ ]:
#coding:utf-8
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

In [10]:
from __future__ import print_function
import numpy as np
import colorama
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


N_HYPEROPT_PROBES = 500
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

# ----------------------------------------------------------

colorama.init()

# ---------------------------------------------------------------------

def get_lgb_params(space):
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
    lgb_params['objective'] = 'regression'
    lgb_params['metric'] = 'rmse'
    lgb_params['learning_rate'] = space['learning_rate']
    lgb_params['num_leaves'] = int(space['num_leaves'])
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
    lgb_params['max_depth'] = -1
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
    lgb_params['feature_fraction'] = space['feature_fraction']
    lgb_params['bagging_fraction'] = space['bagging_fraction']
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
    lgb_params['nthread'] = 4
    return lgb_params

# ---------------------------------------------------------------------

obj_call_count = 0
cur_best_score = 0 # 0 or np.inf
log_writer = open( '../lgb-hyperopt-log.txt', 'w' )


def objective(space):
    global obj_call_count, cur_best_score

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    lgb_params = get_lgb_params(space)

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    print('Params: {}'.format(params_str) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    out_of_fold = np.zeros(len(X_train))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
        D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
        # Train
        num_round = 10000
        clf = lgb.train(lgb_params,
                           D_train,
                           num_boost_round=num_round,
                           # metrics='mlogloss',
                           valid_sets=D_val,
                           # valid_names='val',
                           # fobj=None,
                           # feval=None,
                           # init_model=None,
                           # feature_name='auto',
                           # categorical_feature='auto',
                           early_stopping_rounds=200,
                           # evals_result=None,
                           verbose_eval=False,
                           # learning_rates=None,
                           # keep_training_booster=False,
                           # callbacks=None
                           )
        # predict
        nb_trees = clf.best_iteration
        val_loss = clf.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
        score = r2_score(out_of_fold, Y_train)

    print('val_r2_score={}'.format(score))

    log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
    log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}

# --------------------------------------------------------------------------------

space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1,)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

                                                                                                                       
LightGBM objective call #1 cur_best_score=0.00000
Params: bagging_fraction=0.9495098920597587 bagging_freq=2.0 feature_fraction=0.8169217366799087 lambda_l1=8.406029844506385 lambda_l2=1.3524912332831274 learning_rate=0.0037880965191488905 max_bin=98.0 min_data_in_leaf=28.0 min_sum_hessian_in_leaf=4.282857492623979 num_leaves=77.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.31084761117)])                                                        
nb_trees=9 val_loss=OrderedDict([('rmse', 855077.5893014137)])                                                         
nb_trees=1 val_loss=OrderedDict([('rmse', 10939.460379768701)])                                                        
val_r2_score=-93834.12643501292                                                                                        
                                                                   

nb_trees=1 val_loss=OrderedDict([('rmse', 10918.868039687055)])                                                        
val_r2_score=-90252.8610238328                                                                                         
                                                                                                                       
LightGBM objective call #10 cur_best_score=0.00000
Params: bagging_fraction=0.8614647526411151 bagging_freq=1.0 feature_fraction=0.9345861153044671 lambda_l1=5.157215657732572 lambda_l2=4.3293775442240765 learning_rate=0.00962767555105916 max_bin=93.0 min_data_in_leaf=33.0 min_sum_hessian_in_leaf=8.035794424443393 num_leaves=86.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.94840311457)])                                                        
nb_trees=5 val_loss=OrderedDict([('rmse', 855077.5157203453)])                                                         
nb_trees=1 val_loss=OrderedDict([('rmse', 10960.96533731255)])      

nb_trees=1 val_loss=OrderedDict([('rmse', 425402.97616172355)])                                                        
nb_trees=30 val_loss=OrderedDict([('rmse', 855071.3264936904)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10920.473849433805)])                                                        
val_r2_score=-36714.47384023333                                                                                        
                                                                                                                       
LightGBM objective call #19 cur_best_score=0.00000
Params: bagging_fraction=0.8451192493653001 bagging_freq=14.0 feature_fraction=0.8024340466692942 lambda_l1=0.9358045937834447 lambda_l2=3.2859330574595678 learning_rate=0.004763366061918222 max_bin=92.0 min_data_in_leaf=66.0 min_sum_hessian_in_leaf=1.7110174760760524 num_leaves=80.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.10837444186)]) 

Params: bagging_fraction=0.9868596122238559 bagging_freq=11.0 feature_fraction=0.7568453178452483 lambda_l1=5.571985290089427 lambda_l2=7.207278726463765 learning_rate=0.00741382761433478 max_bin=126.0 min_data_in_leaf=58.0 min_sum_hessian_in_leaf=7.074878413215743 num_leaves=53.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.1678310545)])                                                         
nb_trees=25 val_loss=OrderedDict([('rmse', 855071.5867449263)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10922.999853005154)])                                                        
val_r2_score=-39044.00593325264                                                                                        
                                                                                                                       
LightGBM objective call #28 cur_best_score=0.00000
Params: bagging_fraction=0.9456006853312625 bagging_freq=8.0 featur

val_r2_score=-94376.3218234984                                                                                         
                                                                                                                       
LightGBM objective call #36 cur_best_score=0.00000
Params: bagging_fraction=0.8186821152350072 bagging_freq=7.0 feature_fraction=0.814945780176308 lambda_l1=4.566483713713042 lambda_l2=6.153046143273842 learning_rate=0.008080144125551487 max_bin=135.0 min_data_in_leaf=87.0 min_sum_hessian_in_leaf=4.724867729804865 num_leaves=22.0
nb_trees=7 val_loss=OrderedDict([('rmse', 425402.26947549335)])                                                        
nb_trees=6 val_loss=OrderedDict([('rmse', 855076.0939887451)])                                                         
nb_trees=2 val_loss=OrderedDict([('rmse', 10946.466380288348)])                                                        
val_r2_score=-84429.6884802806                                      

...

nb_trees=359 val_loss=OrderedDict([('rmse', 854995.0497904036)])                                                       
nb_trees=2 val_loss=OrderedDict([('rmse', 10952.885246003058)])                                                        
val_r2_score=-15017.720555926695                                                                                       
                                                                                                                       
LightGBM objective call #461 cur_best_score=0.00000
Params: bagging_fraction=0.8526961973115682 bagging_freq=4.0 feature_fraction=0.8376600664866505 lambda_l1=7.297485797513137 lambda_l2=7.883453728739657 learning_rate=0.008122415149342447 max_bin=118.0 min_data_in_leaf=78.0 min_sum_hessian_in_leaf=7.365558690856036 num_leaves=73.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.25476362125)])                                                        
nb_trees=11 val_loss=OrderedDict([('rmse', 855074.3366932989)])   

Params: bagging_fraction=0.9275622768269736 bagging_freq=13.0 feature_fraction=0.785814170145281 lambda_l1=9.069486920337516 lambda_l2=4.630708346383603 learning_rate=0.009355853570239693 max_bin=88.0 min_data_in_leaf=47.0 min_sum_hessian_in_leaf=4.443779930382391 num_leaves=77.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.1657139863)])                                                         
nb_trees=5 val_loss=OrderedDict([('rmse', 855076.7479799761)])                                                         
nb_trees=1 val_loss=OrderedDict([('rmse', 10922.00169132537)])                                                         
val_r2_score=-80922.9098978547                                                                                         
                                                                                                                       
LightGBM objective call #470 cur_best_score=0.00000
Params: bagging_fraction=0.9097061945119012 bagging_freq=10.0 featu

val_r2_score=-87206.37363116248                                                                                        
                                                                                                                       
LightGBM objective call #478 cur_best_score=0.00000
Params: bagging_fraction=0.9070494400447324 bagging_freq=11.0 feature_fraction=0.7778233539350005 lambda_l1=8.832184901809814 lambda_l2=4.6414744624935045 learning_rate=0.006537779742972099 max_bin=165.0 min_data_in_leaf=43.0 min_sum_hessian_in_leaf=4.98197028606613 num_leaves=96.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.16638673685)])                                                        
nb_trees=33 val_loss=OrderedDict([('rmse', 855072.6744826954)])                                                        
nb_trees=2 val_loss=OrderedDict([('rmse', 10926.055377591685)])                                                        
val_r2_score=-31732.33044579005                                  

nb_trees=24 val_loss=OrderedDict([('rmse', 855075.2100981476)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10927.162151464345)])                                                        
val_r2_score=-38680.00207629866                                                                                        
                                                                                                                       
LightGBM objective call #487 cur_best_score=0.00000
Params: bagging_fraction=0.8987430221056018 bagging_freq=15.0 feature_fraction=0.7958077668757111 lambda_l1=8.464647497123295 lambda_l2=1.0786652969995703 learning_rate=0.007219801074712537 max_bin=150.0 min_data_in_leaf=46.0 min_sum_hessian_in_leaf=4.162346800139888 num_leaves=82.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.1988944873)])                                                         
nb_trees=18 val_loss=OrderedDict([('rmse', 855074.3412744317)]) 

Params: bagging_fraction=0.9255991323248796 bagging_freq=6.0 feature_fraction=0.7510331588142595 lambda_l1=7.86907229893262 lambda_l2=5.593694982964664 learning_rate=0.00790849893555937 max_bin=157.0 min_data_in_leaf=56.0 min_sum_hessian_in_leaf=4.7811637918204175 num_leaves=72.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.1714333882)])                                                         
nb_trees=9 val_loss=OrderedDict([('rmse', 855075.1779122622)])                                                         
nb_trees=1 val_loss=OrderedDict([('rmse', 10923.285548676131)])                                                        
val_r2_score=-74298.71802015328                                                                                        
                                                                                                                       
LightGBM objective call #496 cur_best_score=0.00000
Params: bagging_fraction=0.8976557260955328 bagging_freq=13.0 featu